In [ ]:
import subprocess
import sys

def install_packages():
    packages = [
        "langgraph",
        "langchain",
        "langchain-anthropic",
        "langchain-community",
        "requests",
        "python-dotenv",
        "duckduckgo-search"
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
            print(f"✓ Installed {package}")
        except subprocess.CalledProcessError:
            print(f"✗ Failed to install {package}")

print("Installing required packages...")
install_packages()
print("Installation complete!\n")

In [ ]:
import os
import json
import math
import requests
from typing import Dict, List, Any, Annotated, TypedDict
from datetime import datetime
import operator

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from duckduckgo_search import DDGS

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = "Use Your API Key Here"

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
from typing import TypedDict

class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]

@tool
def calculator(expression: str) -> str:
    """
    Perform mathematical calculations. Supports basic arithmetic, trigonometry, and more.

    Args:
        expression: Mathematical expression as a string (e.g., "2 + 3 * 4", "sin(3.14159/2)")

    Returns:
        Result of the calculation as a string
    """
    try:
        allowed_names = {
            'abs': abs, 'round': round, 'min': min, 'max': max,
            'sum': sum, 'pow': pow, 'sqrt': math.sqrt,
            'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
            'log': math.log, 'log10': math.log10, 'exp': math.exp,
            'pi': math.pi, 'e': math.e
        }

        expression = expression.replace('^', '**')

        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return f"Result: {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

In [ ]:
@tool
def web_search(query: str, num_results: int = 3) -> str:
    """
    Search the web for information using DuckDuckGo.

    Args:
        query: Search query string
        num_results: Number of results to return (default: 3, max: 10)

    Returns:
        Search results as formatted string
    """
    try:
        num_results = min(max(num_results, 1), 10)

        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=num_results))

        if not results:
            return f"No search results found for: {query}"

        formatted_results = f"Search results for '{query}':\n\n"
        for i, result in enumerate(results, 1):
            formatted_results += f"{i}. **{result['title']}**\n"
            formatted_results += f"   {result['body']}\n"
            formatted_results += f"   Source: {result['href']}\n\n"

        return formatted_results
    except Exception as e:
        return f"Error performing web search: {str(e)}"

In [ ]:
@tool
def weather_info(city: str) -> str:
    """
    Get current weather information for a city using OpenWeatherMap API.
    Note: This is a mock implementation for demo purposes.

    Args:
        city: Name of the city

    Returns:
        Weather information as a string
    """
    mock_weather = {
        "new york": {"temp": 22, "condition": "Partly Cloudy", "humidity": 65},
        "london": {"temp": 15, "condition": "Rainy", "humidity": 80},
        "tokyo": {"temp": 28, "condition": "Sunny", "humidity": 70},
        "paris": {"temp": 18, "condition": "Overcast", "humidity": 75}
    }

    city_lower = city.lower()
    if city_lower in mock_weather:
        weather = mock_weather[city_lower]
        return f"Weather in {city}:\n" \
               f"Temperature: {weather['temp']}°C\n" \
               f"Condition: {weather['condition']}\n" \
               f"Humidity: {weather['humidity']}%"
    else:
        return f"Weather data not available for {city}. (This is a demo with limited cities: New York, London, Tokyo, Paris)"

In [ ]:
@tool
def text_analyzer(text: str) -> str:
    """
    Analyze text and provide statistics like word count, character count, etc.

    Args:
        text: Text to analyze

    Returns:
        Text analysis results
    """
    if not text.strip():
        return "Please provide text to analyze."

    words = text.split()
    sentences = text.split('.') + text.split('!') + text.split('?')
    sentences = [s.strip() for s in sentences if s.strip()]

    analysis = f"Text Analysis Results:\n"
    analysis += f"• Characters (with spaces): {len(text)}\n"
    analysis += f"• Characters (without spaces): {len(text.replace(' ', ''))}\n"
    analysis += f"• Words: {len(words)}\n"
    analysis += f"• Sentences: {len(sentences)}\n"
    analysis += f"• Average words per sentence: {len(words) / max(len(sentences), 1):.1f}\n"
    analysis += f"• Most common word: {max(set(words), key=words.count) if words else 'N/A'}"

    return analysis

In [ ]:
@tool
def current_time() -> str:
    """
    Get the current date and time.

    Returns:
        Current date and time as a formatted string
    """
    now = datetime.now()
    return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

In [ ]:
tools = [calculator, web_search, weather_info, text_analyzer, current_time]

def create_llm():
    if ANTHROPIC_API_KEY:
        return ChatAnthropic(
            model="claude-3-haiku-20240307",
            temperature=0.1,
            max_tokens=1024
        )
    else:
        class MockLLM:
            def invoke(self, messages):
                last_message = messages[-1].content if messages else ""

                if any(word in last_message.lower() for word in ['calculate', 'math', '+', '-', '*', '/', 'sqrt', 'sin', 'cos']):
                    import re
                    numbers = re.findall(r'[\d\+\-\*/\.\(\)\s\w]+', last_message)
                    expr = numbers[0] if numbers else "2+2"
                    return AIMessage(content="I'll help you with that calculation.",
                                   tool_calls=[{"name": "calculator", "args": {"expression": expr.strip()}, "id": "calc1"}])
                elif any(word in last_message.lower() for word in ['search', 'find', 'look up', 'information about']):
                    query = last_message.replace('search for', '').replace('find', '').replace('look up', '').strip()
                    if not query or len(query) < 3:
                        query = "python programming"
                    return AIMessage(content="I'll search for that information.",
                                   tool_calls=[{"name": "web_search", "args": {"query": query}, "id": "search1"}])
                elif any(word in last_message.lower() for word in ['weather', 'temperature']):
                    city = "New York"
                    words = last_message.lower().split()
                    for i, word in enumerate(words):
                        if word == 'in' and i + 1 < len(words):
                            city = words[i + 1].title()
                            break
                    return AIMessage(content="I'll get the weather information.",
                                   tool_calls=[{"name": "weather_info", "args": {"city": city}, "id": "weather1"}])
                elif any(word in last_message.lower() for word in ['time', 'date']):
                    return AIMessage(content="I'll get the current time.",
                                   tool_calls=[{"name": "current_time", "args": {}, "id": "time1"}])
                elif any(word in last_message.lower() for word in ['analyze', 'analysis']):
                    text = last_message.replace('analyze this text:', '').replace('analyze', '').strip()
                    if not text:
                        text = "Sample text for analysis"
                    return AIMessage(content="I'll analyze that text for you.",
                                   tool_calls=[{"name": "text_analyzer", "args": {"text": text}, "id": "analyze1"}])
                else:
                    return AIMessage(content="Hello! I'm a multi-tool agent powered by Claude. I can help with:\n• Mathematical calculations\n• Web searches\n• Weather information\n• Text analysis\n• Current time/date\n\nWhat would you like me to help you with?")

            def bind_tools(self, tools):
                return self

        print("⚠️  Note: Using mock LLM for demo. Add your ANTHROPIC_API_KEY for full functionality.")
        return MockLLM()

llm = create_llm()
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def agent_node(state: AgentState) -> Dict[str, Any]:
    """Main agent node that processes messages and decides on tool usage."""
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: AgentState) -> str:
    """Determine whether to continue with tool calls or end."""
    last_message = state["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    return END

In [ ]:
def create_agent_graph():
    tool_node = ToolNode(tools)

    workflow = StateGraph(AgentState)

    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node)

    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
    workflow.add_edge("tools", "agent")

    memory = MemorySaver()

    app = workflow.compile(checkpointer=memory)

    return app

print("Creating LangGraph Multi-Tool Agent...")
agent = create_agent_graph()
print("✓ Agent created successfully!\n")

In [ ]:
def test_agent():
    """Test the agent with various queries."""
    config = {"configurable": {"thread_id": "test-thread"}}

    test_queries = [
        "What's 15 * 7 + 23?",
        "Search for information about Python programming",
        "What's the weather like in Tokyo?",
        "What time is it?",
        "Analyze this text: 'LangGraph is an amazing framework for building AI agents.'"
    ]

    print("🧪 Testing the agent with sample queries...\n")

    for i, query in enumerate(test_queries, 1):
        print(f"Query {i}: {query}")
        print("-" * 50)

        try:
            response = agent.invoke(
                {"messages": [HumanMessage(content=query)]},
                config=config
            )

            last_message = response["messages"][-1]
            print(f"Response: {last_message.content}\n")

        except Exception as e:
            print(f"Error: {str(e)}\n")

In [ ]:
def chat_with_agent():
    """Interactive chat function."""
    config = {"configurable": {"thread_id": "interactive-thread"}}

    print("🤖 Multi-Tool Agent Chat")
    print("Available tools: Calculator, Web Search, Weather Info, Text Analyzer, Current Time")
    print("Type 'quit' to exit, 'help' for available commands\n")

    while True:
        try:
            user_input = input("You: ").strip()

            if user_input.lower() in ['quit', 'exit', 'q']:
                print("Goodbye!")
                break
            elif user_input.lower() == 'help':
                print("\nAvailable commands:")
                print("• Calculator: 'Calculate 15 * 7 + 23' or 'What's sin(pi/2)?'")
                print("• Web Search: 'Search for Python tutorials' or 'Find information about AI'")
                print("• Weather: 'Weather in Tokyo' or 'What's the temperature in London?'")
                print("• Text Analysis: 'Analyze this text: [your text]'")
                print("• Current Time: 'What time is it?' or 'Current date'")
                print("• quit: Exit the chat\n")
                continue
            elif not user_input:
                continue

            response = agent.invoke(
                {"messages": [HumanMessage(content=user_input)]},
                config=config
            )

            last_message = response["messages"][-1]
            print(f"Agent: {last_message.content}\n")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"Error: {str(e)}\n")

In [2]:
if __name__ == "__main__":
    test_agent()

    print("=" * 60)
    print("🎉 LangGraph Multi-Tool Agent is ready!")
    print("=" * 60)

    chat_with_agent()

def quick_demo():
    """Quick demonstration of agent capabilities."""
    config = {"configurable": {"thread_id": "demo"}}

    demos = [
        ("Math", "Calculate the square root of 144 plus 5 times 3"),
        ("Search", "Find recent news about artificial intelligence"),
        ("Time", "What's the current date and time?")
    ]

    print("🚀 Quick Demo of Agent Capabilities\n")

    for category, query in demos:
        print(f"[{category}] Query: {query}")
        try:
            response = agent.invoke(
                {"messages": [HumanMessage(content=query)]},
                config=config
            )
            print(f"Response: {response['messages'][-1].content}\n")
        except Exception as e:
            print(f"Error: {str(e)}\n")

print("\n" + "="*60)
print("🔧 Usage Instructions:")
print("1. Add your ANTHROPIC_API_KEY to use Claude model")
print("   os.environ['ANTHROPIC_API_KEY'] = 'your-anthropic-api-key'")
print("2. Run quick_demo() for a quick demonstration")
print("3. Run chat_with_agent() for interactive chat")
print("4. The agent supports: calculations, web search, weather, text analysis, and time")
print("5. Example: 'Calculate 15*7+23' or 'Search for Python tutorials'")
print("="*60)

Installing required packages...
✓ Installed langgraph
✓ Installed langchain
✓ Installed langchain-anthropic
✓ Installed langchain-community
✓ Installed requests
✓ Installed python-dotenv
✓ Installed duckduckgo-search
Installation complete!

Creating LangGraph Multi-Tool Agent...
✓ Agent created successfully!

🧪 Testing the agent with sample queries...

Query 1: What's 15 * 7 + 23?
--------------------------------------------------
Response: So, 15 * 7 + 23 = 128.

Query 2: Search for information about Python programming
--------------------------------------------------
Response: These search results provide a good overview of Python programming, including the official Python website, Wikipedia article, and a popular tutorial from W3Schools. Let me know if you need any other information!

Query 3: What's the weather like in Tokyo?
--------------------------------------------------
Response: Based on the weather information, it looks like the weather in Tokyo is sunny with a temperature